In [2]:
#r "./src/Model/bin/Debug/net6.0/Model.dll"

open Model


# What is this?

* Have you ever felt the problem with architectural diagrams is they are _too visual_?
* Have you ever thought, "this diagram is great in theory, but what if I could validate it somehow?"
* Have you thought that your diagrams do too little, and by focussing on one aspect of your architecture they are too clear?

## Well, this is the solution

I was looking for a way to visualise a complex architecture, that allowed me to include some often-overlooked concepts as _reliability_ and _ownership_. Not finding a modelling tool that would fit my needs, I turned to what I always turned to - I wrote some code.

This is the result, a simple DSL for describing architectures that can then be used to generate diagrams of a variety of flavours, from a single source of truth.

### No, really, Why?

By creating a DSL in code, we gain two things.

1. We can create complex hierarchies that are difficult to visualise (but still reflective of the truth of the system), and then create simplified visualisations without losing fidelity.
1. We can test our architecture, validate best practices and search for pain points directly from the architecure model.

By modelling the architecture, and not simply diagramming it, we can test our assumptions, validate our behaviours and estimate the reliability and performance of your architecture before you've written a single line of code. (Except for these ones.)

## Enough waffle

You're right, let's do a demonstration.

Imagine you have an architecture, consisting of a single application with no dependencies, that was written by a mythical perfect engineer, and has no bugs. You might model that like so:

In [3]:
let ``my perfect application`` = {
        name = "a totally reliable service"
        serviceType = InternalService
        reliabilityProfile = randomUptimeProfile 1.0
        metadata = None
    }


You can now validate that your perfect application is actually perfect. Using the method `walkService` which runs a test operation against the application.

In [4]:
#r "nuget: Xunit, *-*"

open Xunit

[<Fact>]
let ``Reliable services are always reliable`` () =
    let successes, failures, degradations = determineServiceUptime 1 ``my perfect application``

    Assert.StrictEqual(degradations, 1)

// Run the test, we're in a notebook, not a test runner

``Reliable services are always reliable``()


Installed Packages Xunit, 2.7.0

Error: Xunit.Sdk.StrictEqualException: Assert.StrictEqual() Failure: Values differ
Expected: 0
Actual:   1
   at Xunit.Assert.StrictEqual[T](T expected, T actual) in /_/src/xunit.assert/Asserts/EqualityAsserts.cs:line 971
   at <StartupCode$FSI_0009>.$FSI_0009.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodBaseInvoker.InvokeWithNoArgs(Object obj, BindingFlags invokeAttr)

Oh wait, our test was wrong. Perfect services are always working, not unavailable.

But, you get the picture.

What if our perfect service depends on another service? 

Here we're using the `randomUptimeProfile` function that models a service with a given chance of success.

In [5]:
let ``Our unreliable dependency`` = {
    name = "a totally unreliable service"
    serviceType = InternalService
    reliabilityProfile = randomUptimeProfile 0.5
    metadata = None
}

let ``my perfect application`` = {
        name = "a totally reliable service"
        serviceType = InternalService
        reliabilityProfile = randomUptimeProfile 1.0
        metadata = None
    }

``my perfect application`` |> dependsOn ``Our unreliable dependency``

// This should work fine, right?
let successes, failures, degradations = determineServiceUptime 3 ``my perfect application``
printfn "Successes: %A, Failures: %A, Degradations: %A" successes failures degradations


Successes: 2, Failures: 1, Degradations: 0


Well, that's not looking so good, but let's run it a bunch of times to get more statistically significant data.

In [6]:
let success, failures, _ = determineServiceUptime 1000 ``my perfect application``

sprintf "successes: %d, failures: %d\n" success failures


successes: 511, failures: 489


Oh yes, that makes sense. What would we do in our architecture to improve the reliability? Implement retries with exponential backoff! 

By feeding the `randomUptimeProfile` through the `retrying` strategy we can model a system with retries.

In [7]:
open Reliability.Patterns

let ``Our unreliable dependency`` = {
    name = "a totally unreliable service"
    serviceType = InternalService
    reliabilityProfile = randomUptimeProfile 0.5
    metadata = None
}

let ``my nearly perfect application`` = {
        name = "a totally reliable service"
        serviceType = InternalService
        reliabilityProfile = randomUptimeProfile 1.0
        metadata = None
    }

``my nearly perfect application`` |> dependsOn (``Our unreliable dependency`` |> mitigatedBy (retrying 3))

let success, failures, _ = determineServiceUptime 1000 ``my nearly perfect application``

sprintf "successes: %d, failures: %d\n" success failures


successes: 506, failures: 494


Oh yes, that's much better.